# Deconvolution of the LArSoft opdet simulation

In [83]:
import sys; sys.path.insert(0, '../'); from lib.__init__ import *

*** This notebook is used to analyse the simulated data of the deconvolution tests in larsoft. The output of the workflow is a collection of hits. The hits are then converted into a format that can be compared against the true photon information. ***

In [84]:
# Load data according to type: RAW (digitized), DEC (deconvolved)
max_ev = 50
%time data0 = load_root_new("RAW","../data/Digispe100eve_ideal_10may_hist.root",    ["digispe","wavedigispe"],np.loadtxt("../template/MILANO/FBK/fbk_deco.txt"),max_ev=max_ev,label="IDEAL")
%time data1 = load_root_new("RAW","../data/Digispe100eve_template_10may_hist.root", ["digispe","wavedigispe"],np.loadtxt("../template/MILANO/FBK/fbk_deco.txt"),max_ev=max_ev,label="RAW")
%time data2 = load_root_new("DEC","../data/decon_gauss100eve_10may_hist.root",      ["digispe","opdecohana"], np.loadtxt("../template/MILANO/FBK/fbk_deco.txt"),max_ev=max_ev,label="GAUSS")
%time data3 = load_root_new("DEC","../data/decon_wiener100eve_aucale10may_hist.root",     ["digispe","opdecohana"], np.loadtxt("../template/MILANO/FBK/fbk_deco.txt"),max_ev=max_ev,label="WIENER")

# Select your favorite color map
color_map={data0["LABEL"]: "violet", data1["LABEL"]: "#3366CC", data2["LABEL"]: "#66AA00", data3["LABEL"]:"#FF9900"}
# sandybrown,seagreen,seashell,sienna,silver,skyblue,slateblue,slategray,slategrey,snow,springgreen,steelblue,tan,teal,thistle,tomato,turquoise,violet,wheat,white,whitesmoke,yellow,yellowgreen

Index(['EV', 'CH', 'PE'], dtype='object')


In [ ]:
# Load ophit data for each type
branches = ["Amplitude","EventID","OpChannel","PE","PeakTime"]
%time raw_reco0 = ophit_data("../data/ophitspe_ideal100eve_10may_hist.root",        scaling=1,label="IDEAL",   branches=branches,as_df=False)
%time raw_reco1 = ophit_data("../data/ophitspe_template100eve_10may_hist.root",     scaling=1,label="RAW",     branches=branches,as_df=False)
%time raw_reco2 = ophit_data("../data/ophitspe_gauss100eve_10may_hist.root",        scaling=100,label="GAUSS", branches=branches,as_df=False)
%time raw_reco3 = ophit_data("../data/ophitspe_wiener100eve_aucale10may_hist.root", scaling=100,label="WIENER",branches=branches,as_df=False)


['EventID', 'HitID', 'OpChannel', 'PeakTimeAbs', 'PeakTime', 'Frame', 'Width', 'Area', 'Amplitude', 'PE', 'FastToTotal']
CPU times: user 193 ms, sys: 8 ms, total: 201 ms
Wall time: 200 ms
['EventID', 'HitID', 'OpChannel', 'PeakTimeAbs', 'PeakTime', 'Frame', 'Width', 'Area', 'Amplitude', 'PE', 'FastToTotal']
CPU times: user 192 ms, sys: 16 ms, total: 208 ms
Wall time: 208 ms
['EventID', 'HitID', 'OpChannel', 'PeakTimeAbs', 'PeakTime', 'Frame', 'Width', 'Area', 'Amplitude', 'PE', 'FastToTotal']
CPU times: user 70.4 ms, sys: 0 ns, total: 70.4 ms
Wall time: 70.3 ms
['EventID', 'HitID', 'OpChannel', 'PeakTimeAbs', 'PeakTime', 'Frame', 'Width', 'Area', 'Amplitude', 'PE', 'FastToTotal']
CPU times: user 56 ms, sys: 24 µs, total: 56 ms
Wall time: 73.3 ms


In [ ]:
%time reco0 = order_ophit_data(raw_reco0,max_ev,data0["RECO"]["CH"])
%time reco1 = order_ophit_data(raw_reco1,max_ev,data1["RECO"]["CH"])
%time reco2 = order_ophit_data(raw_reco2,max_ev,data2["RECO"]["CH"])
%time reco3 = order_ophit_data(raw_reco3,max_ev,data3["RECO"]["CH"])

CPU times: user 2.36 s, sys: 0 ns, total: 2.36 s
Wall time: 2.36 s
CPU times: user 2.39 s, sys: 0 ns, total: 2.39 s
Wall time: 2.39 s
CPU times: user 675 ms, sys: 7.95 ms, total: 683 ms
Wall time: 683 ms
CPU times: user 506 ms, sys: 7 µs, total: 506 ms
Wall time: 506 ms


In [ ]:
# Combine data and ophit data
%time ophit_0 = combine_data_ophit(data0,reco0)
%time ophit_1 = combine_data_ophit(data1,reco1)
%time ophit_2 = combine_data_ophit(data2,reco2)
%time ophit_3 = combine_data_ophit(data3,reco3)

CPU times: user 31.8 ms, sys: 0 ns, total: 31.8 ms
Wall time: 31.7 ms
CPU times: user 50.4 ms, sys: 7.98 ms, total: 58.4 ms
Wall time: 58 ms
CPU times: user 51.9 ms, sys: 0 ns, total: 51.9 ms
Wall time: 51.8 ms
CPU times: user 47 ms, sys: 4 ms, total: 51 ms
Wall time: 50.9 ms


*** Visualize individual wvfs according to channel ***

In [ ]:
# Choose between a random channel number or select yourself
# ev = 1; ch = 218; wf = 1
ev = np.random.choice(data1["RECO"]["EV"])
ch = np.random.choice(data1["RECO"]["CH"][data1["RECO"]["EV"] == ev])
wf = np.random.choice(data1["RECO"]["#WVF"][(data1["RECO"]["EV"] == ev)*(data1["RECO"]["CH"] == ch)])

# Find the index of the channel number in the data
try:
    num0 = np.where((data0["RECO"]["EV"] == ev)*(data0["RECO"]["CH"] == ch)*(data0["RECO"]["#WVF"] == wf))[0][0]
except:
    num0 = 0
    print("No ideal data for %s %s %s"%(ev,ch,wf))
    
num1 = np.where((data1["RECO"]["EV"] == ev)*(data1["RECO"]["CH"] == ch)*(data1["RECO"]["#WVF"] == wf))[0][0]

# Genarte figure with subplots
fig = make_subplots(rows=2, cols=2,subplot_titles=('', ''))

# Add waveform traces to the figure
fig.add_trace(go.Scatter(line=dict(color=color_map[data0["LABEL"]]),name="%s #PE: %.2f"%(data1["LABEL"],data0["RECO"]["PE"][num0]),x=1e6*16e-9*(np.linspace(0,1000,1001))+data0["RECO"]["WVF_IX"][num0],y=data0["RECO"]["WVF"][num0]-data0["PEDESTAL"]),col=1,row=1)
fig.add_trace(go.Scatter(line=dict(color=color_map[data1["LABEL"]]),name="%s #PE: %.2f"%(data1["LABEL"],data1["RECO"]["PE"][num1]),x=1e6*16e-9*(np.linspace(0,1000,1001))+data1["RECO"]["WVF_IX"][num1],y=data1["RECO"]["WVF"][num1]-data1["PEDESTAL"]),col=2,row=1)
fig.add_trace(go.Scatter(line=dict(color=color_map[data2["LABEL"]]),name="%s #PE: %.2f"%(data2["LABEL"],data2["RECO"]["PE"][num1]),x=1e6*16e-9*(np.linspace(0,1000,1001))+data2["RECO"]["WVF_IX"][num1],y=data2["RECO"]["WVF"][num1]),col=1,row=2)
fig.add_trace(go.Scatter(line=dict(color=color_map[data3["LABEL"]]),name="%s #PE: %.2f"%(data3["LABEL"],data3["RECO"]["PE"][num1]),x=1e6*16e-9*(np.linspace(0,1000,1001))+data3["RECO"]["WVF_IX"][num1],y=data3["RECO"]["WVF"][num1]),col=2,row=2)

# Add vertical lines according to the reconstructed T0
fig.add_vline(x=data0["RECO"]["T0"][num0], line_width=2, line_dash="dash", line_color="gray",col=1,row=1)
fig.add_vline(x=data1["RECO"]["T0"][num1], line_width=2, line_dash="dash", line_color="gray",col=2,row=1)
fig.add_vline(x=data2["RECO"]["T0"][num1], line_width=2, line_dash="dash", line_color="gray",col=1,row=2)
fig.add_vline(x=data3["RECO"]["T0"][num1], line_width=2, line_dash="dash", line_color="gray",col=2,row=2)

# Add true PE times to the figure
# if len(data0["TRUE"]["PETIMES"][num0]) > 100:
# fig.add_trace(go.Histogram(name="TRUE #PE: %.2f"%(len(data0["TRUE"]["PETIMES"][num0])),x=np.asarray(data0["TRUE"]["PETIMES"][num0])),col=1,row=1)
# fig.add_trace(go.Histogram(name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num1])),x=np.asarray(data1["TRUE"]["PETIMES"][num1])),col=2,row=1)
# else:
fig.add_trace(go.Scatter(marker_symbol="triangle-up",mode="markers",line=dict(color="black"),name="TRUE #PE: %.2f"%(len(data0["TRUE"]["PETIMES"][num0])),x=np.asarray(data0["TRUE"]["PETIMES"][num0]),y=np.zeros(len(data0["TRUE"]["PETIMES"][num0]))),col=1,row=1)
fig.add_trace(go.Scatter(marker_symbol="triangle-up",mode="markers",line=dict(color="black"),name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num1])),x=np.asarray(data1["TRUE"]["PETIMES"][num1]),y=np.zeros(len(data1["TRUE"]["PETIMES"][num1]))),col=2,row=1)
fig.add_trace(go.Scatter(marker_symbol="triangle-up",mode="markers",line=dict(color="black"),name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num1])),x=np.asarray(data1["TRUE"]["PETIMES"][num1]),y=np.zeros(len(data1["TRUE"]["PETIMES"][num1]))),col=1,row=2)
fig.add_trace(go.Scatter(marker_symbol="triangle-up",mode="markers",line=dict(color="black"),name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num1])),x=np.asarray(data1["TRUE"]["PETIMES"][num1]),y=np.zeros(len(data1["TRUE"]["PETIMES"][num1]))),col=2,row=2)

# Add ophits to the figure
fig.add_trace(go.Scatter(name="OPHITFINDER #PE: %.2f"%(ophit_0["PE"][num0]),line=dict(color="#DC3912"),mode="markers",x=ophit_0["TIMES"][num0],y=ophit_0["AMP"][num0]/reco0["SCALING"]),col=1,row=1)
fig.add_trace(go.Scatter(name="OPHITFINDER #PE: %.2f"%(ophit_1["PE"][num1]),line=dict(color="#DC3912"),mode="markers",x=ophit_1["TIMES"][num1],y=ophit_1["AMP"][num1]/reco1["SCALING"]),col=2,row=1)
fig.add_trace(go.Scatter(name="OPHITFINDER #PE: %.2f"%(ophit_2["PE"][num1]),line=dict(color="#DC3912"),mode="markers",x=ophit_2["TIMES"][num1],y=ophit_2["AMP"][num1]/reco2["SCALING"]),col=1,row=2)
fig.add_trace(go.Scatter(name="OPHITFINDER #PE: %.2f"%(ophit_3["PE"][num1]),line=dict(color="#DC3912"),mode="markers",x=ophit_3["TIMES"][num1],y=ophit_3["AMP"][num1]/reco3["SCALING"]),col=2,row=2)

# Update the layout
fig.update_layout(title="Comparison for ev %i, ch %i and wvf %i"%(ev,ch,wf),xaxis_title="Time in [&mu;s]",yaxis_title="Amp. in [ADC]",xaxis2_title="Time in [&mu;s]",yaxis2_title="Amp. in [a.u.]",xaxis3_title="Time in [&mu;s]",yaxis3_title="Amp. in [a.u.]",xaxis4_title="Time in [&mu;s]",yaxis4_title="Amp. in [a.u.]")
fig.update_layout(autosize=True,height=600,font=dict(size=14))
fig.update_layout(template="presentation")
# fig.update_yaxes(type="log")
fig.show()

No ideal data for 2 12 1


*** CONFIGURE FILTER FOR ANALYSIS ***

In [ ]:
with np.errstate(divide='ignore', invalid='ignore'):
    pe_error_0 = (np.asarray(data0["RECO"]["PE"])-np.asarray(data0["TRUE"]["PE"]))/np.asarray(data0["TRUE"]["PE"])
    pe_error_1 = (np.asarray(data1["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
    pe_error_2 = (np.asarray(data2["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
    pe_error_3 = (np.asarray(data3["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
    ophitpe_error_0 = (np.asarray(ophit_0["PE"])-np.asarray(data0["TRUE"]["PE"]))/np.asarray(data0["TRUE"]["PE"])
    ophitpe_error_1 = (np.asarray(ophit_1["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
    ophitpe_error_2 = (np.asarray(ophit_2["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
    ophitpe_error_3 = (np.asarray(ophit_3["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])

t0_reco_0 = np.asarray(data0["RECO"]["T0"])-np.asarray(data0["TRUE"]["T0"])
t0_reco_1 = np.asarray(data1["RECO"]["T0"])-np.asarray(data1["TRUE"]["T0"])
t0_reco_2 = np.asarray(data2["RECO"]["T0"])-np.asarray(data1["TRUE"]["T0"])
t0_reco_3 = np.asarray(data3["RECO"]["T0"])-np.asarray(data1["TRUE"]["T0"])

amp           = np.concatenate([data0["RECO"]["AMP"],data1["RECO"]["AMP"],data2["RECO"]["AMP"],data3["RECO"]["AMP"]])
pe_reco       = np.concatenate([data0["RECO"]["PE"], data1["RECO"]["PE"], data2["RECO"]["PE"], data3["RECO"]["PE"]])
pe_true       = np.concatenate([data0["TRUE"]["PE"], data1["TRUE"]["PE"], data1["TRUE"]["PE"], data1["TRUE"]["PE"]])
pe_ophit      = np.concatenate([ophit_0["PE"],       ophit_1["PE"],       ophit_2["PE"],       ophit_3["PE"]])
ophitpe_error = np.concatenate([ophitpe_error_0,     ophitpe_error_1,     ophitpe_error_2,     ophitpe_error_3])
pe_error      = np.concatenate([pe_error_0,          pe_error_1,          pe_error_2,          pe_error_3])
t0_reco       = np.concatenate([t0_reco_0,           t0_reco_1,           t0_reco_2,           t0_reco_3])
filter_label  = np.concatenate([[data0["LABEL"]]*len(pe_error_0),[data1["LABEL"]]*len(pe_error_1),[data2["LABEL"]]*len(pe_error_2),[data3["LABEL"]]*len(pe_error_3)])

df = pd.DataFrame({"FILTER":filter_label, "ERROR PE":pe_error, "ERROR OPHIT PE":ophitpe_error, "TRUE PE":pe_true, "RECO PE":pe_reco, "OPHIT PE":pe_ophit, "AMP [a.u.]":amp, "RECO T0":t0_reco})

In [ ]:
fig = px.scatter(data_frame=df[(df['TRUE PE'] > 10)*(df['TRUE PE'] < 1000)], 
                x="AMP [a.u.]",
                y='TRUE PE',
                color='FILTER',
                color_discrete_map=color_map,
                # marginal_x='box',
                trendline="ols")
fig.update_layout(autosize=True,height=800,font=dict(size=16),template="presentation")
fig.show()

results = px.get_trendline_results(fig) # print(results)
results.query("FILTER == 'GAUSS'").px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                 1.023e+05
Date:                Mon, 15 May 2023   Prob (F-statistic):               0.00
Time:                        12:02:52   Log-Likelihood:                -4194.8
No. Observations:                 926   AIC:                             8394.
Df Residuals:                     924   BIC:                             8403.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         14.1185      0.971     14.542      0.000      12.213      16.024
x1            25.5435      0.080    319.903      0.000      25.387      25.700
==============================================================================
Omnibus:                      149.096   Durbin-Watson:                   1.777
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1485.377
Skew:                          -0.383   Prob(JB):                         0.00
Kurtosis:                       9.157   Cond. No.                         16.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
fig = px.scatter(data_frame=df[(df['TRUE PE'] > 10)*(df['TRUE PE'] < 2000)], 
                x="ERROR OPHIT PE",
                y="TRUE PE",
                color='FILTER',
                color_discrete_map=color_map,
                marginal_x='box',
                # marginal_y='histogram',
                template="presentation")
fig.update_layout(autosize=True,height=800,font=dict(size=16))
fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
xlim = (-1,1); acc = 1000; p0=[10,0,1e-2]

fig = gauss_fit_distribution(df[(df["TRUE PE"] > 400)*(df["TRUE PE"] < 1000)],"ERROR OPHIT PE","FILTER",color_map,xlim,acc,p0)
fig.update_layout(bargap = 0, xaxis = dict(range=xlim),
                # yaxis = dict(range= [0,50])
                )
fig.show()

----------------IDEAL----------------
AMP	= 1.86E+01	± 1.95E-01
MEAN	= -5.03E-03	± 8.80E-05
STD	= 7.25E-03	± 8.80E-05
-------------------------------------
----------------RAW----------------
AMP	= 9.86E-01	± 2.66E-06
MEAN	= -2.84E-09	± 1.01E+00
STD	= 2.26E-04	± 2.28E-01
-------------------------------------
----------------GAUSS----------------
AMP	= 6.79E+00	± 1.42E-01
MEAN	= -1.99E-03	± 4.70E-04
STD	= 1.94E-02	± 4.70E-04
-------------------------------------
----------------WIENER----------------
AMP	= 6.83E+00	± 1.15E-01
MEAN	= 2.32E-04	± 3.72E-04
STD	= 1.91E-02	± 3.72E-04
-------------------------------------


In [ ]:
fig = px.histogram(data_frame=df[(df['TRUE PE'] > 0)*(df['TRUE PE'] < 1000)],
                    x="RECO T0",
                    color='FILTER',
                    color_discrete_map=color_map,
                    barmode="overlay",
                    marginal='box',
                    template="presentation",
                    nbins=1000,
                    range_x=[-0.16,0.32])
fig.update_layout(autosize=True,height=800,font=dict(size=16))
fig.update_layout(bargap=0)
fig.show()

In [ ]:
min_wvf_length = 500
ave_wvfs = []
for data in [data2,data3]:

    # Select only the waveforms with a minimum length
    data["RECO"]["SHORT_WVF"] = []
    for wvf in data["RECO"]["WVF"]:
        if len(wvf) > min_wvf_length:
            data["RECO"]["SHORT_WVF"].append(wvf[:min_wvf_length])

    # Prepare the averaged deconvolved waveforms for the scintillation profile fit
    ave_wvf = np.mean(np.asarray(data["RECO"]["SHORT_WVF"]),axis=0)
    ave_wvf = ave_wvf/np.max(ave_wvf)
    ave_wvf = ave_wvf[np.argmax(ave_wvf):]
    ave_wvfs.append(ave_wvf)
    
    # Fit the scintillation profile of the averaged deconvolved waveforms
    initial = [0.7,0.3,6,1400]
    labels = ["CONSTANT","AMPLITUDE","TAU FAST","TAU SLOW"]
    popt, pcov = curve_fit(scint_profile,16*np.arange(len(ave_wvf)),ave_wvf,p0=initial)
    perr = np.sqrt(np.diag(pcov))

    # Print the fit results
    ave_wvfs.append(scint_profile(16*np.arange(len(ave_wvf)),*popt))
    print("\n----------- FIT VALUES " + data["LABEL"]+ " ------------")
    for i in range(len(initial)):
        print("%s:\t%.2E\t%.2E"%(labels[i], popt[i], perr[i]))
    print("-----------------------------------------")


----------- FIT VALUES GAUSS ------------
CONSTANT:	1.82E+02	1.97E+00
AMPLITUDE:	1.84E-01	3.24E-03
TAU FAST:	7.12E+01	1.27E+00
TAU SLOW:	1.68E+03	3.66E+01
-----------------------------------------

----------- FIT VALUES WIENER ------------
CONSTANT:	1.89E+02	2.01E+00
AMPLITUDE:	1.78E-01	3.01E-03
TAU FAST:	7.13E+01	1.22E+00
TAU SLOW:	1.82E+03	3.84E+01
-----------------------------------------


In [ ]:
# Generate the dataframe for the plot
wvfs  = np.concatenate([ave_wvfs[0],ave_wvfs[1],ave_wvfs[2],ave_wvfs[3]])
wvfs_x  = np.concatenate([16*np.arange(len(ave_wvf)),16*np.arange(len(ave_wvf)),16*np.arange(len(ave_wvf)),16*np.arange(len(ave_wvf))])
fit_label  = np.concatenate([["WVF"]*len(ave_wvf),["FIT"]*len(ave_wvf),["WVF"]*len(ave_wvf),["FIT"]*len(ave_wvf)])
filter_label  = np.concatenate([[data2["LABEL"]]*len(ave_wvf),[data2["LABEL"]]*len(ave_wvf),[data3["LABEL"]]*len(ave_wvf),[data3["LABEL"]]*len(ave_wvf)])

fit_df = pd.DataFrame({"WVF":wvfs,"TIME in [ns]":wvfs_x,"FILTER":filter_label,"FIT":fit_label})

# Plot the results
fig = px.line(data_frame=fit_df,x="TIME in [ns]",y="WVF",color="FIT",facet_col="FILTER",log_y=True,color_discrete_map=color_map)
fig.show()